In [9]:
import pandas as pd
import numpy as np
import os
import pyreadr


In [10]:
dir_data = "C:/Users/chris/Documents/Projects/Names/Data/"

In [11]:
# Initialize an empty list to store dataframes
dfs = []

# Loop through each file in the directory
for year in range(1900, 2024):
    file_path = os.path.join(dir_data, f"raw/yob{year}.txt")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, header=None)
        df['year'] = year
        dfs.append(df)

# Concatenate all dataframes
all_data = pd.concat(dfs, ignore_index=True)
all_data.columns = ['Name', 'Gender', 'Count', 'Year']
all_data

,Name,Gender,Count,Year
0,Mary,F,16705,1900
1,Helen,F,6342,1900
2,Anna,F,6114,1900
3,Margaret,F,5304,1900
4,Ruth,F,4765,1900
...,...,...,...,...
2064950,Zyell,M,5,2023
2064951,Zyen,M,5,2023
2064952,Zymirr,M,5,2023
2064953,Zyquan,M,5,2023


In [12]:
# Pivot the data to get separate columns for male and female counts
pivot_data = all_data.pivot_table(index=['Name', 'Year'], columns='Gender', values='Count', aggfunc='sum', fill_value=0).reset_index()

# Rename the columns
pivot_data.columns = ['Name', 'Year', 'Count_Female', 'Count_Male']

# Calculate the total count
pivot_data['Count_Total'] = pivot_data['Count_Female'] + pivot_data['Count_Male']

# Calculate the rank for female and male names
pivot_data['Rank_Female'] = pivot_data.groupby('Year')['Count_Female'].rank(ascending=False, method='min')
pivot_data['Rank_Male'] = pivot_data.groupby('Year')['Count_Male'].rank(ascending=False, method='min')

# Calculate the proportion of each name for female and male
pivot_data['Rate_Female'] = pivot_data['Count_Female'] / pivot_data.groupby('Year')['Count_Female'].transform('sum')
pivot_data['Rate_Male'] = pivot_data['Count_Male'] / pivot_data.groupby('Year')['Count_Male'].transform('sum')

# Calculate the yearly share of female and male names
pivot_data['Percent_Female'] = pivot_data['Count_Female'] / pivot_data['Count_Total']
pivot_data['Percent_Male'] = pivot_data['Count_Male'] / pivot_data['Count_Total']

# Sort the data by Year and then by Count_Total in descending order
pivot_data = pivot_data.sort_values(by=['Year', 'Count_Total'], ascending=[True, False])

pivot_data

,Name,Year,Count_Female,Count_Male,Count_Total,Rank_Female,Rank_Male,Rate_Female,Rate_Male,Percent_Female,Percent_Male
1223786,Mary,1900,16705,75,16780,1.0,230.0,0.055723,0.000498,0.995530,0.004470
868794,John,1900,46,9829,9875,499.0,1.0,0.000153,0.065320,0.004658,0.995342
1823156,William,1900,44,8579,8623,509.0,2.0,0.000147,0.057013,0.005103,0.994897
790651,James,1900,41,7245,7286,524.0,3.0,0.000137,0.048148,0.005627,0.994373
711149,Helen,1900,6342,18,6360,2.0,578.0,0.021155,0.000120,0.997170,0.002830
...,...,...,...,...,...,...,...,...,...,...,...
1882627,Zyen,2023,0,5,5,17534.0,12190.0,0.000000,0.000003,0.000000,1.000000
1883644,Zymirr,2023,0,5,5,17534.0,12190.0,0.000000,0.000003,0.000000,1.000000
1883650,Zyn,2023,5,0,5,15121.0,14150.0,0.000003,0.000000,1.000000,0.000000
1883855,Zyquan,2023,0,5,5,17534.0,12190.0,0.000000,0.000003,0.000000,1.000000


In [13]:
min_year = 1960

# Filter the data for years since min_year
filtered_data = all_data[all_data['Year'] >= min_year]

# Pivot the data to get separate columns for male and female counts
overall_data = filtered_data.pivot_table(index='Name', columns='Gender', values='Count', aggfunc='sum', fill_value=0).reset_index()

# Rename the columns
overall_data.columns = ['Name', 'Count_Female', 'Count_Male']

# Calculate the total count
overall_data['Count_Total'] = overall_data['Count_Female'] + overall_data['Count_Male']

# Calculate the rank for female and male names
overall_data['Rank_Female'] = overall_data['Count_Female'].rank(ascending=False, method='min')
overall_data['Rank_Male'] = overall_data['Count_Male'].rank(ascending=False, method='min')

# Calculate the proportion of each name for female and male
overall_data['Rate_Female'] = overall_data['Count_Female'] / overall_data['Count_Female'].sum()
overall_data['Rate_Male'] = overall_data['Count_Male'] / overall_data['Count_Male'].sum()

# Calculate the share of female and male names
overall_data['Percent_Female'] = overall_data['Count_Female'] / overall_data['Count_Total']
overall_data['Percent_Male'] = overall_data['Count_Male'] / overall_data['Count_Total']

# Sort the data by Count_Total in descending order
overall_data = overall_data.sort_values(by='Count_Total', ascending=False)

overall_data

,Name,Count_Female,Count_Male,Count_Total,Rank_Female,Rank_Male,Rate_Female,Rate_Male,Percent_Female,Percent_Male
61877,Michael,17132,3098647,3115779,909.0,1.0,1.538617e-04,2.595628e-02,0.005498,0.994502
21219,David,8832,2145109,2153941,1433.0,2.0,7.931978e-05,1.796882e-02,0.004100,0.995900
37479,James,10306,2082283,2092589,1294.0,3.0,9.255770e-05,1.744255e-02,0.004925,0.995075
41864,John,8237,1973259,1981496,1496.0,4.0,7.397611e-05,1.652930e-02,0.004157,0.995843
17447,Christopher,8973,1955983,1964956,1416.0,5.0,8.058609e-05,1.638458e-02,0.004567,0.995433
...,...,...,...,...,...,...,...,...,...,...
43194,Juliamae,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000
43192,Juliahnna,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000
43189,Juliagrace,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000
65527,Nebeyu,0,5,5,65242.0,34931.0,0.000000e+00,4.188324e-08,0.000000,1.000000


In [14]:
# Read the RData file
name_race_probs = pyreadr.read_r(dir_data+'first_nameRaceProbs.rData')
name_race_probs = name_race_probs['first_nameRaceProbs']

# Clean up the name column
name_race_probs['name'] = name_race_probs['name'].str.strip().str.title()

# Merge with overall_data with indicator set to True
merged_data = overall_data.merge(name_race_probs, left_on='Name', right_on='name', how='left', indicator=True)

# Print the indicator column's value_counts
print(merged_data['_merge'].value_counts())

# Drop the indicator column
merged_data = merged_data.drop(columns=['_merge'])
# Rename the race columns
merged_data = merged_data.rename(columns={'whi': 'pr_white', 'bla': 'pr_black', 'his': 'pr_hispanic', 'asi': 'pr_asian'})

# Drop the 'oth' and 'name' columns
merged_data = merged_data.drop(columns=['oth', 'name'])
merged_data


_merge
both          64949
left_only     30733
right_only        0
Name: count, dtype: int64


,Name,Count_Female,Count_Male,Count_Total,Rank_Female,Rank_Male,Rate_Female,Rate_Male,Percent_Female,Percent_Male,pr_white,pr_black,pr_hispanic,pr_asian
0,Michael,17132,3098647,3115779,909.0,1.0,1.538617e-04,2.595628e-02,0.005498,0.994502,0.817958,0.134655,0.029259,0.006104
1,David,8832,2145109,2153941,1433.0,2.0,7.931978e-05,1.796882e-02,0.004100,0.995900,0.843589,0.093486,0.044733,0.006973
2,James,10306,2082283,2092589,1294.0,3.0,9.255770e-05,1.744255e-02,0.004925,0.995075,0.828457,0.153643,0.006438,0.003470
3,John,8237,1973259,1981496,1496.0,4.0,7.397611e-05,1.652930e-02,0.004157,0.995843,0.867057,0.099637,0.018107,0.005635
4,Christopher,8973,1955983,1964956,1416.0,5.0,8.058609e-05,1.638458e-02,0.004567,0.995433,0.781474,0.160660,0.037080,0.006736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95677,Juliamae,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000,0.173184,0.787709,0.000000,0.000000
95678,Juliahnna,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000,NaN,NaN,NaN,NaN
95679,Juliagrace,5,0,5,55998.0,40721.0,4.490477e-08,0.000000e+00,1.000000,0.000000,NaN,NaN,NaN,NaN
95680,Nebeyu,0,5,5,65242.0,34931.0,0.000000e+00,4.188324e-08,0.000000,1.000000,NaN,NaN,NaN,NaN


In [15]:
# Save pivot_data to parquet
pivot_data.to_parquet(os.path.join(dir_data, 'clean/name_year_data.parquet'))

# Save merged_data to parquet
merged_data.to_parquet(os.path.join(dir_data, 'clean/name_overall_data.parquet'))
merged_data.to_parquet(os.path.join(dir_data, 'clean/name_overall_data.csv'))